# Projet Data Mining 2020-2021

#### Spotify Dataset, 1921 - 2020, 160k+ Tracks

*Source* : [Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data.csv)

El-Haddad Amal  
Berhocoïrigoin Elena

**Introduction** : L'objectif de ce projet est de mettre en oeuvre une approche complète de fouille de données depuis le prétraitement des données jusqu'à l'évaluation de la méthode de fouille. 

## 1 . Spotify Dataset

In [2]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
warnings.filterwarnings('ignore')

### 1.1 Présentation des données

In [33]:
#Charger le dataset :
spotify = pd.read_csv('./data.csv',sep=",",header=0)
#Afficher les dimensions du dataframe ainsi que les trois premières lignes :
print("Dimensions : ", spotify.shape)
spotify.head(3)

Dimensions :  (170653, 19)


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339


In [57]:
#Vérifier le type des variables et la présence ou non de valeurs manquantes :
types = pd.DataFrame(spotify.dtypes, columns = ["Types"])
NaN = pd.DataFrame(spotify.isna().any(), columns = ["na"])
#Récupérer le min et le max des variables pour lesquelles ça fait sens :
Max = pd.DataFrame(spotify.drop(columns = ["artists", "id", "name", "release_date"]).max(axis=0), columns = ["max"])
Min = pd.DataFrame(spotify.drop(columns = ["artists", "id", "name", "release_date"]).min(axis=0), columns = ["min"])
#Rassembler les valeurs dans un seul data frame :
result = pd.merge(types, NaN, left_index = True, right_index = True, how="outer")
result = pd.merge(result, Max, left_index = True, right_index = True, how="outer")
result = pd.merge(result, Min, left_index = True, right_index = True, how="outer")
#Afficher le résultat :
print(result)

                    Types     na          max     min
acousticness      float64  False        0.996     0.0
artists            object  False          NaN     NaN
danceability      float64  False        0.988     0.0
duration_ms         int64  False  5403500.000  5108.0
energy            float64  False        1.000     0.0
explicit            int64  False        1.000     0.0
id                 object  False          NaN     NaN
instrumentalness  float64  False        1.000     0.0
key                 int64  False       11.000     0.0
liveness          float64  False        1.000     0.0
loudness          float64  False        3.855   -60.0
mode                int64  False        1.000     0.0
name               object  False          NaN     NaN
popularity          int64  False      100.000     0.0
release_date       object  False          NaN     NaN
speechiness       float64  False        0.970     0.0
tempo             float64  False      243.507     0.0
valence           float64  F

Jettons un coup d'oeil aux 19 variables que nous avons dans notre jeu de données :

**Variables numériques** :  
- <span style="color: #26B260"> **acousticness** </span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette valeur décrit le degré d'"acoustique" de la chanson. Un score de 1.0 signifie qu'il est fort probable que le titre soit une musique acoustique, autrement dit qu'aucun instrument électronique moderne n'ait été employé (ex : guitare électrique, violon électrique, synthétiseur, ordinateur, sampleur ...)
- <span style="color: #26B260">**danceability**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Une valeur de 1 signifie que le titre est aproprié pour danser, une valeur de 0, qu'il est difficile de danser sur cette musique. Le score s'appuie sur une combinaison d'éléments, dont le tempo, la stabilité du rythme, la régularité générale.
- <span style="color: #26B260">**energy**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette valeur rend compte de l'intensité et de l'activité perçue d'un titre. Typiquement, une musique énergique (score proche de 1.0) aura tendance à être rapide, sonore et bruyante. Le death metal aura ainsi une valeur élevée, contrairement à un prélude de Bach par exemple. Les caractéristiques perceptuelles contribuant à cet attribut comprennent la gamme dynamique, l'intensité sonore perçue, le timbre, l'attaque et l'entropie générale.
- <span style="color: #26B260">**duration_ms**</span> <span style="color: #6ac580">Entier variant généralement entre 200k et 300k</span> : Durée du morceau en millisecondes.
- <span style="color: #26B260">**instrumentalness**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette valeur rend compte de la présence ou non de chant, et de la place qu'il occupe dans la musique. Les sons "Ooh" et "aah" sont considérés comme "instrumentaux" dans ce contexte. Plus le score est proche de 1.0, plus il est probable que la piste ne contienne pas de contenu vocal (>0.5). Les morceaux de rap auront ainsi de faible valeur pour cette mesure, contrairement à de la musique classique par exemple.
- <span style="color: #26B260">**valence**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette valeur décrit la "positivité musicale" du morceau. Les titres à forte valence sonnent plus positif (i.e heureux, joyeux, euphorique), tandis que les morceaux à faible valence sonnent plus négatif (i.e triste, déprimé, énervé).
- <span style="color: #26B260">**popularity**</span> <span style="color: #6ac580">Intervalle de 0 à 100</span> : Popularité du morceau. Elle est calculée par un algorithme et est basée, en grande partie, sur le nombre total de lectures du titre et sur leur caractère récent (i.e les chansons qui sont beaucoup écoutées aujourd'hui auront un score plus élevé que les pistes qui ont été beaucoup jouées par le passé).
- <span style="color: #26B260">**tempo**</span> <span style="color: #6ac580">Flottant variant généralement entre 50 et 150</span> : Tempo global estimé d'un morceau en battements par minute (BPM). Dans la terminologie musicale, le tempo est la vitesse ou le rythme d'un morceau donné et découle directement de la durée moyenne des battements.
- <span style="color: #26B260">**liveness**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette valeur décrit la probabilité que la chanson ait été enregistrée avec un public en direct. Une valeur supérieure à 0,8 signifie qu'il est fort probable que le morceau ait été enregistré en live.
- <span style="color: #26B260">**loudness**</span> <span style="color: #6ac580">Flottant variant généralement entre -60 et 0 db</span> : Intensité sonore globale d'une piste en décibels (dB). Les valeurs d'intensité sonore sont calculées en moyenne sur l'ensemble du morceau.
- <span style="color: #26B260">**speechiness**</span> <span style="color: #6ac580">Intervalle de 0 à 1</span> : Cette mesure détecte la présence de mots parlés dans une piste. Plus l'enregistrement est exclusivement vocal (par exemple talk-show, livre audio, poésie), plus la valeur de l'attribut est proche de 1,0. Les valeurs supérieures à 0,66 décrivent des pistes qui sont probablement entièrement constituées de mots parlés. Les valeurs entre 0,33 et 0,66 décrivent des pistes qui peuvent contenir à la fois de la musique et de la parole (y compris rap). Les valeurs inférieures à 0,33 représentent des pistes probablement musicales et d'autres pistes non vocales. 
- <span style="color: #26B260">**year**</span> <span style="color: #6ac580">Intervalle de 1921 à 2020</span> : Année de sortie du titre.
  
**Variables catégorielles**:    
- <span style="color: #26B260">**mode**</span> <span style="color: #6ac580">0 = Mineur, 1 = Majeur</span> : Mode - mineur ou majeur - du morceau. Détermine la couleur du morceau.
- <span style="color: #26B260">**explicit**</span> <span style="color: #6ac580">0 = Pas de contenu explicite - ou inconnu, 1 = Contenu explicite</span> : Présence de paroles explicites ou non.
- <span style="color: #26B260">**key**</span> <span style="color: #6ac580"> Entier variant de 0 à 11 </span> : Tonalité estimée du morceau. Toutes les tonalités sont codées comme des valeurs allant de 0 à 11 selon l'échelle des hauteurs standard, en commancant par 0 : Do (C) ; 1 : Do♯/Ré♭ (C♯/D♭) ; 2 : Ré (D) [etc](https://fr.wikipedia.org/wiki/Hauteur_(musique)). Si aucune tonalité n'a été détectée, la valeur est -1, ce n'est pas le cas dans notre jeu de données.
- <span style="color: #26B260">**artists**</span> : Liste des artistes mentionnées.
- <span style="color: #26B260">**release_date**</span> : Date de sortie au format yyyy-mm-dd, la plupart du temps. Toutefois, la précision peut varier d'un titre à l'autre (seulement yyyy ou yyyy-mm).
- <span style="color: #26B260">**name**</span> : Nom de la chanson.
- <span style="color: #26B260">**id**</span> : Id Spotify de la piste.

In [ ]:
#Retirer la colonne "id" qui ne sera pas utile pour l'analyse :
spotify.drop(columns="id", inplace = True)

Le mois de sortie du titre pourrait avoir une influence sur sa popularité (ex : sortie durant l'été). De la variable "release_date" nous ne garderons donc que cette information lorsqu'elle est disponible (puisque nous avons déjà une colonne pour l'année de sortie) :

In [ ]:
#Récupérer le mois de sortie du titre :
import re
re.sub( expression à chercher, par quoi la remplacer (\<numero du groupe> par
ex), la chaîne d’origine )
#Supprimer la colonne "release_date" désormais inutile :

### 1.2 Statistiques et Analyses Préliminaires